In [ ]:
# import csv
import pandas as pd

train = pd.read_csv('length_weight.csv')

length = train.iloc[1::,0]
length_list = length.values.tolist()
x_train = torch.tensor(x_values).reshape(-1, 1)

weight = train.iloc[1::,1]
weight_list = length.values.tolist()
y_train = torch.tensor(y_values).reshape(-1, 1)

# downscale
scale = 10**(-2)
x_train * scale
y_train * scale

# x_values = []
# y_values = []

# with open('length_weight.csv') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
#     next(spamreader)
    
#     for row in spamreader:
#         x_values.append(float(row[0])*scale)
#         y_values.append(float(row[1])*scale)

# x_train = torch.tensor(x_values).reshape(-1, 1)  # x_train = [[1], [1.5], [2], [3], [4], [5], [6]]
# y_train = torch.tensor(y_values).reshape(-1, 1)  # y_train = [[5], [3.5], [3], [4], [3], [1.5], [2]]

# print(x_train)
# print(y_train)
print("")

In [ ]:
# Observed/training input and output
# x_train = torch.tensor([1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0]).reshape(-1, 1)  # x_train = [[1], [1.5], [2], [3], [4], [5], [6]]
# y_train = torch.tensor([5.0, 3.5, 3.0, 4.0, 3.0, 1.5, 2.0]).reshape(-1, 1)  # y_train = [[5], [3.5], [3], [4], [3], [1.5], [2]]

# print(x_train)
# print(y_train)

In [ ]:
from IPython.display import display, clear_output
import time

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1) 


def show_plot():
    # correct for downscaling
    x_train_tmp = x_train / scale
    y_train_tmp = y_train / scale

    # clear from previous
    plt.figure().clear()

    # Visualize result
    plt.plot(x_train_tmp, y_train_tmp, 'o', label='$(x^{(i)},y^{(i)})$')

    plt.xlabel('x')
    plt.ylabel('y')
    x = torch.tensor([[torch.min(x_train_tmp)], [torch.max(x_train_tmp)]])  # x = [[1], [6]]]

    plt.plot(x, model.f(x).detach() - (8), label='$\\hat y = f(x) = xW+b$')
    # plt.legend()
    
    clear_output(wait=True)
    display(plt.gcf())
    time.sleep(0.05)

    # clear_output(wait = True)
    # plt.pause(0.5)

In [ ]:
import torch
import csv
import matplotlib.pyplot as plt


# Observed/training input and output
# x_train = torch.tensor([1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0]).reshape(-1, 1)  # x_train = [[1], [1.5], [2], [3], [4], [5], [6]]
# y_train = torch.tensor([5.0, 3.5, 3.0, 4.0, 3.0, 1.5, 2.0]).reshape(-1, 1)  # y_train = [[5], [3.5], [3], [4], [3], [1.5], [2]]


class LinearRegressionModel:
    def __init__(self):
        # Model variables
        self.W = torch.tensor([[0.0]], requires_grad=True)  # requires_grad enables calculation of gradients
        self.b = torch.tensor([[0.0]], requires_grad=True)

    # Predictor
    def f(self, x):
        return x @ self.W + self.b  # @ corresponds to matrix multiplication

    # Uses Mean Squared Error
    def loss(self, x, y):
        return torch.nn.functional.mse_loss(self.f(x), y)
        # return torch.mean(torch.square(self.f(x) - y))  # Can also use torch.nn.functional.mse_loss(self.f(x), y) to possibly increase numberical stability


model = LinearRegressionModel()

print("(start)   W = %s, b = %s, loss = %s" % (model.W, model.b, model.loss(x_train, y_train)))

loss_list = []
loss_count = []

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.SGD([model.W, model.b], 0.5)
for epoch in range(120):
    model.loss(x_train, y_train).backward()  # Compute loss gradients

    # print("(compute) W = %s, b = %s, loss = %s" % (model.W, model.b, model.loss(x_train, y_train)))

    loss_list.append(model.loss(x_train, y_train).item())
    loss_count.append(epoch)

    optimizer.step()  # Perform optimization by adjusting W and b,
    # similar to:
    # model.W -= model.W.grad * 0.01
    # model.b -= model.b.grad * 0.01

    optimizer.zero_grad()  # Clear gradients for next step

    show_plot()

# Print model variables and loss
print("W = %s, b = %s, loss = %s" % (model.W, model.b/scale, model.loss(x_train, y_train)))

# print(loss_list)
# print(loss_count)

# plt.plot(loss_list)
# plt.yscale('log')
# plt.ylabel('some numbers')
# plt.show()

show_plot()
# ------------

# # correct for downscaling
# x_train = x_train / scale
# y_train = y_train / scale

# # Visualize result
# plt.plot(x_train, y_train, 'o', label='$(x^{(i)},y^{(i)})$')

# plt.xlabel('x')
# plt.ylabel('y')
# x = torch.tensor([[torch.min(x_train)], [torch.max(x_train)]])  # x = [[1], [6]]]

# plt.plot(x, model.f(x).detach() - (8), label='$\\hat y = f(x) = xW+b$')
# plt.legend()
# plt.show()

# print("x train")
# print((x_train)[0:10])
# print((x_train/scale)[0:10])